In [ ]:
! pip3 install -q -U pandas shapely geopandas matplotlib

In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon

# Step 1: Obtain the coordinates of the random shape
# Load the CSV file
csv_file = 'mapgeop.csv'  # Replace with your CSV file path
data = pd.read_csv(csv_file, delimiter='\,')

# Load the GeoJSON file
geojson_file = 'Addresses.geojson'  # Replace with your GeoJSON file path
geo_data = gpd.read_file(geojson_file)

# Function to find the coordinate of an address in the GeoJSON
def find_coordinates(address, geo_data):
    for feature in geo_data.iterfeatures():
        if feature['properties']['SBL_ID'] == address:
            return feature['geometry']['coordinates']
    return None

# Extract and match addresses
data['Coordinates'] = data.apply(lambda row: find_coordinates(f"{row['ID']}", geo_data), axis=1)
# drop na values
data = data.dropna(subset=['Coordinates'])
shape_coordinates = data['Coordinates']


# Step 2: Create a polygon geometry from the shape coordinates
shape_polygon = Polygon(shape_coordinates)

# Step 3: Obtain address data
address_data = gpd.read_file("Addresses.geojson")

# Step 4: Perform spatial join to select addresses within the random shape
addresses_in_shape = gpd.sjoin(address_data, gpd.GeoDataFrame(geometry=[shape_polygon]), op="within")
addresses_in_shape.to_csv("selected_addresses.csv", index=False)

print("Addresses within the selected polygon exported successfully!")